In [1]:
import gurobipy as gp

In [6]:
phi = {0:1, 3:1}
optimal  = dict()
z = []
for keys in phi.keys():
    z.append(keys)

for i in range(10):
    try:
        j = 0
        while j + 1 < len(z):
            m = gp.Model()
            m.setParam('OutputFlag', 0)
            x = m.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY, name = 'x')
            y = m.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY, name = 'y')
        # Enter constraints and objective functions
            constr_lower = [x-y >= 1, y <= 0, x+y <= 2]     ############################
            constr_upper = [x >= 0, x <= 3]                 ############################# 
            F_upper = (x-1.5)**2 + (y+1)**2                 #############################
            f_lower = -y                                    #############################
            for cs in constr_lower:
                m.addConstr(cs)
            for cs in constr_upper:
                m.addConstr(cs)
#lagrangian = mu_1*phi[z[0]] + mu_2*phi[z[1]] - lambda_1*mu_1 - lambda_2*mu_2 + beta*(mu_1 + mu_2 - 1) + gamma*(mu_1*z[0] + mu_2*z[1])
            mu_1 = m.addVar(name = "mu_1", ub = gp.GRB.INFINITY)
            mu_2 = m.addVar(name = "mu_2", ub = gp.GRB.INFINITY)
            lambda_1 = m.addVar(lb = 0, ub = gp.GRB.INFINITY)
            lambda_2 = m.addVar(lb = 0, ub = gp.GRB.INFINITY)
            beta = m.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY)
            gamma = m.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY)
            m.addConstr(phi[z[j]] - lambda_1 + beta + gamma*z[j] == 0)
            m.addConstr(phi[z[j+1]] - lambda_2 + beta + gamma*z[j+1] == 0)
            m.addConstr(mu_1 + mu_2 == 1)
            m.addConstr(mu_1*z[j] + mu_2*z[j+1] == x)
            m.addConstr(f_lower <= mu_1*phi[z[j]] +mu_2*phi[z[j+1]])
            m.setObjective(F_upper, gp.GRB.MINIMIZE)
            m.optimize()
            F_upper_value = F_upper.getValue()
            f_lower_value = f_lower.getValue()                  # Value of f at the new point
            z.append(x.X)
#Calculate Phi
            model = gp.Model()
            model.setParam('OutputFlag', 0)
            x = model.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY, name = 'x')
            y = model.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY, name = 'y')
# Enter constraints and objective functions
            x = z[len(z)-1]
            constr_lower = [x-y >= 1, y <= 0, x+y <= 2]
            constr_upper = [x >= 0, x <= 3]
            F_upper = (x-1.5)**2 + (y+1)**2
            f_lower = -y
            
            for cs in constr_lower:
                model.addConstr(cs)
            for cs in constr_upper:
                model.addConstr(cs)
            model.addConstr(x >= z[j])
            model.addConstr(x <= z[j+1])
            model.setObjective(f_lower, gp.GRB.MINIMIZE)
            model.optimize()
#Use optimal objective
            phi_value = f_lower.getValue()                           #Value of Phi at the new point
# Append to dictionary and sort dictionar and the array z
            phi[x] = phi_value
# Also add termination condition given in the paper
            if (phi_value == f_lower_value):
                optimal[(x, y.X)] = F_upper_value
        
            phi = dict(sorted(phi.items()))
            z.sort()
#j+2. j+3
            j = j + 2
    except Exception:
        continue

In [7]:
phi

{0: 1,
 0.7498378678871643: 0.25016213211283567,
 0.7498501928692424: 0.25014980713075763,
 0.7498635313358962: 0.2501364686641038,
 0.7498649353268177: 0.25013506467318225,
 0.7498802443914057: 0.25011975560859434,
 0.7498817220872046: 0.2501182779127954,
 0.7498828101361872: 0.2501171898638128,
 0.7498832049172982: 0.2501167950827018,
 0.749902224204308: 0.25009777579569203,
 0.7499038103771073: 0.2500961896228927,
 0.749904995946659: 0.25009500405334095,
 0.7499054266020088: 0.25009457339799124,
 0.7499063170545438: 0.2500936829454562,
 0.7499066397886051: 0.25009336021139494,
 0.7499069136172233: 0.2500930863827767,
 0.7499070805799849: 0.2500929194200151,
 0.7499332773370462: 0.2500667226629538,
 0.7499351842887326: 0.2500648157112674,
 0.7499364994310328: 0.2500635005689672,
 0.7499371438440201: 0.2500628561559799,
 0.7499381358665733: 0.2500618641334267,
 0.7499384958981551: 0.2500615041018449,
 0.7499388981339352: 0.2500611018660648,
 0.7499391585088838: 0.2500608414911162,
 0.

In [8]:
optimal

{(1.500000000057013, 0.0): 1.0,
 (0.7499411973934422, -0.2500588026065578): 1.1250000069154928,
 (0.7782918133845993, -0.2217081866154007): 1.1266008534091778,
 (0.7499974017221963, -0.2500025982778037): 1.1250000000135025,
 (0.7782918130731187, -0.22170818692688132): 1.1266008533739285,
 (0.7782918136960799, -0.22170818630392009): 1.126600853444427,
 (2.221708200551444, -0.2217082005514439): 1.126600851832074,
 (0.7499948408918544, -0.2500051591081456): 1.1250000000532332,
 (0.7499990075950437, -0.2500009924049563): 1.1250000000019698,
 (0.7782918140075605, -0.22170818599243947): 1.1266008534796765,
 (0.8121659583668205, -0.18783404163317952): 1.1327292127593305,
 (2.1878339867316816, -0.1878339867316816): 1.1327292264113538,
 (2.2217082002388606, -0.22170820023886062): 1.1266008518674486,
 (0.7499923137573745, -0.2500076862426255): 1.1250000001181566,
 (0.7499985791476093, -0.25000142085239074): 1.1250000000040374,
 (0.7498817220872046, -0.2501182779127954): 1.1250000279793295,
 (0.7